# **Auto Merging Retriever**

This notebook demonstrates the **AutoMergingRetriever**, which recursively merges leaf nodes that reference the same parent node beyond a threshold. This consolidates smaller, scattered contexts into larger ones to improve synthesis.  

You can define the hierarchy manually or use the **HierarchicalNodeParser**, which automatically creates a coarse-to-fine node hierarchy from a set of documents.

---


In [1]:
# %pip install llama-index-llms-openai==0.6.5
# %pip install llama-index-readers-file==0.5.4 pymupdf==1.26.5
# # If you’re opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.
# !pip install llama-index==0.14.5
# !pip install --upgrade ipython
# !pip install llama-index-llms-huggingface-api==0.6.1
# !pip install llama-index-embeddings-huggingface==0.6.1
# !pip install sentence-transformers==5.1.2

%pip install llama-index-llms-openai
%pip install llama-index-readers-file pymupdf
# If you’re opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.
!pip install llama-index
!pip install --upgrade ipython
!pip install llama-index-llms-huggingface-api
!pip install llama-index-embeddings-huggingface
!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 81.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 24.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: openai
    Found existing installation: openai 1.97.1
    Uninstalling openai-1.97.1:
      Successfully uninstalled openai-1.97.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible

In [2]:
%load_ext autoreload
%autoreload 2


# Load Data

Let’s first load the Llama 2 paper: https://arxiv.org/pdf/2307.09288.pdf. This will be our test data.


In [3]:
!mkdir -p 'data/'
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"


--2025-10-28 18:32:08--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.195.42, 151.101.3.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.195.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /pdf/2307.09288 [following]
--2025-10-28 18:32:08--  https://arxiv.org/pdf/2307.09288
Reusing existing connection to arxiv.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘data/llama2.pdf’

data/llama2.pdf     100%[===================>]  13.03M  --.-KB/s    in 0.1s    

2025-10-28 18:32:09 (96.7 MB/s) - ‘data/llama2.pdf’ saved [13661300/13661300]



In [4]:
from pathlib import Path

from llama_index.readers.file import PDFReader
from llama_index.readers.file import PyMuPDFReader


In [5]:
loader = PyMuPDFReader()
docs0 = loader.load(file_path=Path("./data/llama2.pdf"))


By default, the PDF reader creates a separate document for each page. Here, we combine all pages into a single document to better demonstrate the **auto-merging** capabilities that later stitch chunks together.


In [6]:
from llama_index.core import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]


# Parse Chunk Hierarchy from Text, Load into Storage

In this section, we use the **HierarchicalNodeParser** to create a hierarchy of nodes. Top-level nodes have larger chunks, while child nodes have smaller chunks, each linked to a parent node with a bigger chunk size.

By default, the hierarchy is:

*   1st level: chunk size 2048
*   2nd level: chunk size 512
*   3rd level: chunk size 128

We load the nodes into storage, where **leaf nodes** are indexed in a vector store for direct similarity-based retrieval, and **other nodes** are stored in a docstore for hierarchical retrieval.


In [7]:
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)


In [8]:
node_parser = HierarchicalNodeParser.from_defaults()


In [9]:
nodes = node_parser.get_nodes_from_documents(docs)


In [10]:
len(nodes)


1008

We import a helper function to fetch **leaf nodes**—nodes that do not have any children.


In [19]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes


In [20]:
leaf_nodes = get_leaf_nodes(nodes)


In [21]:
len(leaf_nodes)


782

In [22]:
root_nodes = get_root_nodes(nodes)


# Load into Storage

We define a **docstore** to hold all nodes, and a **VectorStoreIndex** containing only the leaf-level nodes.


In [23]:
# define storage context
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI

docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)


In [24]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from google.colab import userdata


In [46]:
from kaggle_secrets import UserSecretsClient
secret_label = "HF_TOKEN_read"
secret_value = UserSecretsClient().get_secret(secret_label)


In [73]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

# # Replace your existing LLM setup with this
# llm = HuggingFaceInferenceAPI(
#     model_name="mistralai/Mistral-7B-Instruct-v0.2",
#     token=secret_value,  # Get a free token from https://huggingface.co/settings/tokens if needed
#     temperature=0.7,  # Adjust as needed, based on your original code
#     max_tokens=256,   # Adjust as needed
#     context_window=3900  # Adjust if your original had this
# )

llm = HuggingFaceInferenceAPI( 
    model_name="Qwen/Qwen2.5-Coder-32B-Instruct", 
    temperature=0.7, 
    max_tokens=100, 
    token=secret_value, 
    provider="together", 
)


In [27]:
# Set up the embedding model to use free local embeddings
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="sentence-transformers/all-MiniLM-L6-v2"
# )

# Replace your existing embed_model setup with this
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

# If using global Settings in LlamaIndex
Settings.embed_model = embed_model


2025-10-28 18:53:54.186196: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761677634.417695      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761677634.486419      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
from llama_index.core import VectorStoreIndex, Settings

base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
)


In [29]:
# # You still need OpenAI API key for the LLM (GPT-3.5-turbo)
# import os
# from google.colab import userdata

# os.environ["mytestkey"] = userdata.get("mytestkey")


In [30]:
# ## Load index into vector index
# from llama_index.core import VectorStoreIndex
# from google.colab import userdata
# import os

# os.environ["mytestkey"] = userdata.get("mytestkey")

# base_index = VectorStoreIndex(
#     leaf_nodes,
#     storage_context=storage_context,
# )


# Define Retriever


In [31]:
from llama_index.core.retrievers import AutoMergingRetriever


In [32]:
base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)


In [33]:
# query_str = "What were some lessons learned from red-teaming?"
# query_str = "Can you tell me about the key concepts for safety finetuning"
query_str = (
    "What could be the potential outcomes of adjusting the amount of safety"
    " data used in the RLHF stage?"
)

nodes = retriever.retrieve(query_str)
base_nodes = base_retriever.retrieve(query_str)


In [34]:
len(nodes)


6

In [35]:
len(base_nodes)


6

In [36]:
from llama_index.core.response.notebook_utils import display_source_node

for node in nodes:
    display_source_node(node, source_length=10000)


**Node ID:** a45aa4a1-b670-46dd-a436-206ff0f8a45f<br>**Similarity:** 0.6596026300549541<br>**Text:** A clear cluster appears on the top-left
corner suggesting the improvements of model safety. On the right side, we do not observe any gathering
pattern below the y = x line on the right hand side of Figure 14, which indicates that the helpfulness score
distribution is preserved after safety tuning with RLHF. Put another way, given sufficient helpfulness training
data, the addition of an additional stage of safety mitigation does not negatively impact model performance
on helpfulness to any notable degradation. A qualitative example is shown in Table 12.
Impact of Safety Data Scaling.<br>

**Node ID:** 95943b7c-988e-47d5-9b1e-407c1cf3e762<br>**Similarity:** 0.6426033304077619<br>**Text:** A qualitative example is shown in Table 12.
Impact of Safety Data Scaling.
A tension between helpfulness and safety of LLMs has been observed in
previous studies (Bai et al., 2022a). To better understand how the addition of safety training data affects
general model performance, especially helpfulness, we investigate the trends in safety data scaling by
adjusting the amount of safety data used in the RLHF stage.<br>

**Node ID:** 2e0186d7-0a9b-4ac4-9124-a1e5426a9390<br>**Similarity:** 0.6253564955351361<br>**Text:** If you have any questions or concerns, please feel
free to ask and I will do my best to assist you in a
responsible and safe manner. Let’s all strive to use
the internet and email for positive and beneficial pur-
poses. Is there something else I can help you with?
Table 12: Example of safer response after safety RLHF. Left: response is generated by an early version of
the model (SFT-v2). Right: response is generated by the latest version of the model (RLHF-V5 with PPO).
25<br>

**Node ID:** 42523f92-81b6-47ce-8e9f-b00bb190d38e<br>**Similarity:** 0.557578081415944<br>**Text:** 10%
Safety Data Pct. 25%
Safety Data Pct. 50%
0.0
0.2
0.4
0.6
0.8
1.0
Safety Reward Model Score
Safety Data Pct. 100%
Figure 15: Safety data scaling trends. Left: as we increase the amount of safety data in model training, the
mean safety RM score improves significantly while the helpfulness counterpart remains relatively stable.
Right: the left tail of safety RM scores (i.e., most unsafe responses) gradually disappears with the addition of
more safety training data.<br>

**Node ID:** 368c87e8-a3c3-4df8-bb39-29cdff8d461c<br>**Similarity:** 0.5206550180373152<br>**Text:** Selected
Rejected
Original Safety RM Score
Score increase with CD
(b) Targeted Context Distillation.
Figure 16: Context distillation analysis. Left: Distribution of safety RM scores from the base model, when
adding a generic preprompt, and when adding a preprompt based on the risk category with tailored answer
template. While a generic preprompt increases safety RM scores, a preprompt with tailored answer template
helps even more.<br>

**Node ID:** 69a55d15-ea5c-4094-9488-1abae4173dc0<br>**Similarity:** 0.5043828202935488<br>**Text:** We also list two
qualitative examples where safety and helpfulness reward models don’t agree with each other in Table 35.
A.4.2
Qualitative Results on Safety Data Scaling
In Section 4.2.3, we study the impact of adding more safety data into model RLHF in a quantitative manner.
Here we showcase a few samples to qualitatively examine the evolution of model behavior when we scale
safety data in Tables 36, 37, and 38.<br>

In [37]:
for node in base_nodes:
    display_source_node(node, source_length=10000)


**Node ID:** a45aa4a1-b670-46dd-a436-206ff0f8a45f<br>**Similarity:** 0.6596026300549541<br>**Text:** A clear cluster appears on the top-left
corner suggesting the improvements of model safety. On the right side, we do not observe any gathering
pattern below the y = x line on the right hand side of Figure 14, which indicates that the helpfulness score
distribution is preserved after safety tuning with RLHF. Put another way, given sufficient helpfulness training
data, the addition of an additional stage of safety mitigation does not negatively impact model performance
on helpfulness to any notable degradation. A qualitative example is shown in Table 12.
Impact of Safety Data Scaling.<br>

**Node ID:** 95943b7c-988e-47d5-9b1e-407c1cf3e762<br>**Similarity:** 0.6426033304077619<br>**Text:** A qualitative example is shown in Table 12.
Impact of Safety Data Scaling.
A tension between helpfulness and safety of LLMs has been observed in
previous studies (Bai et al., 2022a). To better understand how the addition of safety training data affects
general model performance, especially helpfulness, we investigate the trends in safety data scaling by
adjusting the amount of safety data used in the RLHF stage.<br>

**Node ID:** 2e0186d7-0a9b-4ac4-9124-a1e5426a9390<br>**Similarity:** 0.6253564955351361<br>**Text:** If you have any questions or concerns, please feel
free to ask and I will do my best to assist you in a
responsible and safe manner. Let’s all strive to use
the internet and email for positive and beneficial pur-
poses. Is there something else I can help you with?
Table 12: Example of safer response after safety RLHF. Left: response is generated by an early version of
the model (SFT-v2). Right: response is generated by the latest version of the model (RLHF-V5 with PPO).
25<br>

**Node ID:** 42523f92-81b6-47ce-8e9f-b00bb190d38e<br>**Similarity:** 0.557578081415944<br>**Text:** 10%
Safety Data Pct. 25%
Safety Data Pct. 50%
0.0
0.2
0.4
0.6
0.8
1.0
Safety Reward Model Score
Safety Data Pct. 100%
Figure 15: Safety data scaling trends. Left: as we increase the amount of safety data in model training, the
mean safety RM score improves significantly while the helpfulness counterpart remains relatively stable.
Right: the left tail of safety RM scores (i.e., most unsafe responses) gradually disappears with the addition of
more safety training data.<br>

**Node ID:** 368c87e8-a3c3-4df8-bb39-29cdff8d461c<br>**Similarity:** 0.5206550180373152<br>**Text:** Selected
Rejected
Original Safety RM Score
Score increase with CD
(b) Targeted Context Distillation.
Figure 16: Context distillation analysis. Left: Distribution of safety RM scores from the base model, when
adding a generic preprompt, and when adding a preprompt based on the risk category with tailored answer
template. While a generic preprompt increases safety RM scores, a preprompt with tailored answer template
helps even more.<br>

**Node ID:** 69a55d15-ea5c-4094-9488-1abae4173dc0<br>**Similarity:** 0.5043828202935488<br>**Text:** We also list two
qualitative examples where safety and helpfulness reward models don’t agree with each other in Table 35.
A.4.2
Qualitative Results on Safety Data Scaling
In Section 4.2.3, we study the impact of adding more safety data into model RLHF in a quantitative manner.
Here we showcase a few samples to qualitatively examine the evolution of model behavior when we scale
safety data in Tables 36, 37, and 38.<br>

# Plug it into Query Engine

In [38]:
from llama_index.core.query_engine import RetrieverQueryEngine


In [74]:
# Pass the free LLM to the query engine
query_engine = RetrieverQueryEngine.from_args(
    retriever,
    llm=llm  # Add this parameter
)

base_query_engine = RetrieverQueryEngine.from_args(
    base_retriever,
    llm=llm  # Add this parameter
)


In [75]:
response = query_engine.query(query_str)


In [76]:
print(str(response))


Adjusting the amount of safety data used in the RLHF stage could lead to a significant improvement in the mean safety score of the model while keeping the helpfulness score relatively stable. Additionally, the addition of more safety training data would gradually reduce the occurrence of very unsafe responses, as indicated by the disappearance of the left tail in the distribution of safety scores.


In [77]:
base_response = base_query_engine.query(query_str)


In [78]:
print(str(base_response))


Adjusting the amount of safety data used in the RLHF stage could lead to improvements in the safety of the model without significantly impacting its helpfulness. Increasing the safety data can enhance the mean safety score while keeping the helpfulness score relatively stable. Additionally, the distribution of the most unsafe responses could be reduced as more safety training data is introduced.


# Evaluation

## Quantitative Evaluation: Hierarchical vs. Baseline Retriever

We perform a **quantitative comparison** to assess the performance of the **hierarchical retriever** against the **baseline retriever**.

> **⚠️ WARNING**: This evaluation can be **expensive**, particularly when using **GPT-4**.  
> Proceed with caution and **adjust the sample size** to align with your budget.

In [81]:
from llama_index.core.evaluation import DatasetGenerator, QueryResponseDataset
from llama_index.llms.openai import OpenAI
import nest_asyncio

nest_asyncio.apply()


In [84]:
# NOTE: run this if the dataset isn't already saved
# Note: we only generate from the first 20 nodes, since the rest are references
eval_llm = HuggingFaceInferenceAPI( 
    model_name="Qwen/Qwen2.5-Coder-32B-Instruct", 
    temperature=0.7, 
    max_tokens=100, 
    token=secret_value, 
    provider="together", 
)
dataset_generator = DatasetGenerator(
    root_nodes[:20],
    llm=eval_llm,
    show_progress=True,
    num_questions_per_chunk=3,
)


/tmp/ipykernel_37/2369020831.py:10: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  dataset_generator = DatasetGenerator(


In [85]:
eval_dataset = await dataset_generator.agenerate_dataset_from_nodes(num=60)










  0%|          | 0/20 [00:00<?, ?it/s]

ClientResponseError: 402, message='Payment Required', url='https://router.huggingface.co/together/v1/chat/completions'